In [ ]:
import pandas as pd, numpy as np
import gc
import torch

# INFERENCE

### Tom

In [ ]:
gc.collect()
torch.cuda.empty_cache()

!cp ../input/kkillerfpzeff2ndplace/fpzeff/notebooks/final/tom_feedback2-stacking_103881768_v29.ipynb  .
!jupyter nbconvert --execute --to notebook tom_feedback2-stacking_103881768_v29.ipynb
!mv submission.csv submission_tom.csv

In [ ]:
! head submission_tom.csv

### Shujun

In [ ]:
gc.collect()
torch.cuda.empty_cache()


!cp ../input/kkillerfpzeff2ndplace/fpzeff/notebooks/final/shujun_fb-5th-rd-pl_103829488_v11.ipynb .
!jupyter nbconvert --execute --to notebook shujun_fb-5th-rd-pl_103829488_v11.ipynb
!mv submission.csv submission_shujun.csv

In [ ]:
! head submission_shujun.csv

### Kkiller

In [ ]:
import os, re

v_transformers = !pip show transformers
v_transformers = "\n".join(v_transformers)
v_transformers = re.search(r"version\:\s*([\d\.]+)", v_transformers.lower()).group(1).strip()
print("initial transformers version:", v_transformers)
if v_transformers != "4.20.1":
#     os.system('pip show transformers')
    os.system('pip uninstall -y transformers')
    os.system('pip uninstall -y tokenizers')
    os.system('rm -r /opt/conda/lib/python3.7/site-packages/transformers')
    os.system('python -m pip install --no-index --find-links=../input/pip-wheels-transformers-4-20-1/ transformers')
    os.system('python -m pip install --no-index --find-links=../input/pip-wheels-transformers-4-20-1/ tokenizers')
    
    
    import tokenizers
    import transformers
    
    print(f"tokenizers.__version__: {tokenizers.__version__}")
    print(f"transformers.__version__: {transformers.__version__}")
else:
    print("transformers is already up to date")

In [ ]:
gc.collect()
torch.cuda.empty_cache()

!cp ../input/kkillerfpzeff2ndplace/fpzeff/notebooks/final/kkiller-fpzeff-2nd-place-inference_104902394_v2.ipynb .
!jupyter nbconvert --execute --to notebook kkiller-fpzeff-2nd-place-inference_104902394_v2.ipynb
!mv submission.csv submission_kkiller.csv

In [ ]:
! head submission_kkiller.csv

# ENSEMBLE

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
sub_params = [("submission_tom.csv", 0.25), ("submission_shujun.csv", 0.45), ("submission_kkiller.csv", 0.30)]

S = sum(param[1] for param in sub_params)
assert abs(S-1.0) < 1e-3, S

sample_sub = pd.read_csv("../input/feedback-prize-effectiveness/sample_submission.csv")
cols = ["Ineffective", "Adequate", "Effective"]

sub = None
for sub_path, w  in sub_params:
    temp = pd.read_csv(sub_path)
    temp = sample_sub[["discourse_id"]].merge(temp, on="discourse_id", how="left")
    temp[cols] = temp[cols].fillna(1/3.)
    temp[cols] *= w
    
    if sub is None:
        sub = temp
    else:
        sub[cols] += temp[cols]

sub[["discourse_id", *cols]].to_csv("submission.csv", index=False)

print(sub.shape)

sub.head(30)